In [11]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [13]:
import networkx as nx
from typing import List, Tuple, Dict

# 1. Data Loading and Parsing Functions

def load_edges(filename: str) -> List[Tuple[int, int]]:
    """
    Reads the edges from the nodeId.edges file and returns a list of tuples representing each edge.

    Parameters:
    - filename: Path to the edges file.

    Returns:
    - List of tuples where each tuple represents an edge (source, target).
    """
    edges = []
    try:
        with open(filename, 'r') as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) == 2:
                    source, target = map(int, parts)
                    edges.append((source, target))
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
    except Exception as e:
        print(f"An error occurred while loading edges: {e}")
    return edges

def load_circles(filename: str) -> Dict[str, List[int]]:
    """
    Reads the circles from the nodeId.circles file and returns a dictionary with circle names as keys
    and lists of node IDs as values.

    Parameters:
    - filename: Path to the circles file.

    Returns:
    - Dictionary mapping circle names to lists of node IDs.
    """
    circles = {}
    try:
        with open(filename, 'r') as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) >= 2:
                    circle_name = parts[0]
                    node_ids = list(map(int, parts[1:]))
                    circles[circle_name] = node_ids
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
    except Exception as e:
        print(f"An error occurred while loading circles: {e}")
    return circles

def load_features(filename: str) -> Dict[int, List[int]]:
    """
    Reads the node features from the nodeId.feat file and returns a dictionary where each node ID
    maps to a list of binary features.

    Parameters:
    - filename: Path to the features file.

    Returns:
    - Dictionary mapping node IDs to their feature vectors.
    """
    features = {}
    try:
        with open(filename, 'r') as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) >= 2:
                    node_id = int(parts[0])
                    feature_vector = list(map(int, parts[1:]))
                    features[node_id] = feature_vector
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
    except Exception as e:
        print(f"An error occurred while loading features: {e}")
    return features

def load_ego_features(filename: str) -> List[int]:
    """
    Reads the features of the ego node from nodeId.egofeat and returns a list of binary features
    for the ego node.

    Parameters:
    - filename: Path to the ego features file.

    Returns:
    - List of binary features for the ego node.
    """
    try:
        with open(filename, 'r') as file:
            for line in file:
                features = list(map(int, line.strip().split()))
                return features  # Assuming only one line exists
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
    except Exception as e:
        print(f"An error occurred while loading ego features: {e}")
    return []

def load_feature_names(filename: str) -> List[str]:
    """
    Reads the feature names from nodeId.featnames and returns a list of anonymized feature names.

    Parameters:
    - filename: Path to the feature names file.

    Returns:
    - List of feature names.
    """
    feature_names = []
    try:
        with open(filename, 'r') as file:
            for line in file:
                name = line.strip()
                if name:
                    feature_names.append(name)
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
    except Exception as e:
        print(f"An error occurred while loading feature names: {e}")
    return feature_names

# 2. Graph Construction Functions

def construct_graph(edges: List[Tuple[int, int]], ego_node: int) -> nx.Graph:
    """
    Builds the graph using NetworkX, including the ego node connected to all nodes in the edges list.

    Parameters:
    - edges: List of tuples representing edges (source, target).
    - ego_node: The ID of the ego node.

    Returns:
    - A NetworkX graph with the ego node and all edges.
    """
    G = nx.Graph()
    G.add_node(ego_node, label='ego')  # Labeling the ego node if needed
    G.add_edges_from(edges)
    return G

def add_circles_to_graph(graph: nx.Graph, circles: Dict[str, List[int]]) -> None:
    """
    Adds circle information as node attributes for further community analysis.

    Parameters:
    - graph: The NetworkX graph to which circle information will be added.
    - circles: Dictionary mapping circle names to lists of node IDs.

    Returns:
    - None. The graph is modified in place.
    """
    for circle_name, node_ids in circles.items():
        for node_id in node_ids:
            if node_id in graph.nodes:
                if 'circles' in graph.nodes[node_id]:
                    graph.nodes[node_id]['circles'].append(circle_name)
                else:
                    graph.nodes[node_id]['circles'] = [circle_name]
            else:
                # Optionally, add the node if it doesn't exist
                graph.add_node(node_id, circles=[circle_name])
    # Optionally, handle the ego node if it's part of any circle

# 3. Network Metric Calculation Functions

def calculate_degree(graph: nx.Graph) -> Dict[int, int]:
    """
    Calculates the degree for each node and returns a dictionary mapping node IDs to their degree values.

    Parameters:
    - graph: The NetworkX graph.

    Returns:
    - Dictionary mapping node IDs to their degree.
    """
    return dict(graph.degree())

def calculate_clustering_coefficient(graph: nx.Graph) -> Dict[int, float]:
    """
    Computes the clustering coefficient for each node.

    Parameters:
    - graph: The NetworkX graph.

    Returns:
    - Dictionary mapping node IDs to their clustering coefficient.
    """
    return nx.clustering(graph)

def calculate_centrality(graph: nx.Graph, method: str = 'betweenness') -> Dict[int, float]:
    """
    Calculates centrality measures based on the specified method.

    Parameters:
    - graph: The NetworkX graph.
    - method: The centrality measure to calculate ('betweenness', 'closeness', etc.).

    Returns:
    - Dictionary mapping node IDs to their centrality values.

    Raises:
    - ValueError: If an unsupported centrality method is specified.
    """
    if method == 'betweenness':
        return nx.betweenness_centrality(graph)
    elif method == 'closeness':
        return nx.closeness_centrality(graph)
    elif method == 'eigenvector':
        try:
            return nx.eigenvector_centrality(graph, max_iter=1000)
        except nx.PowerIterationFailedConvergence:
            print("Eigenvector centrality did not converge. Returning empty dictionary.")
            return {}
    else:
        raise ValueError(f"Unsupported centrality method: {method}")

# 4. Utility Functions for Printing

def print_degree_statistics(degrees: Dict[int, int]) -> None:
    """
    Prints statistics about node degrees.

    Parameters:
    - degrees: Dictionary mapping node IDs to their degree.
    """
    degree_values = list(degrees.values())
    print("\n--- Degree Statistics ---")
    print(f"Total Nodes: {len(degrees)}")
    print(f"Average Degree: {sum(degree_values)/len(degree_values):.2f}")
    print(f"Minimum Degree: {min(degree_values)}")
    print(f"Maximum Degree: {max(degree_values)}")
    print(f"Median Degree: {sorted(degree_values)[len(degree_values)//2]}")
    print("\nTop 5 Nodes by Degree:")
    top_degree = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:5]
    for node, degree in top_degree:
        print(f"Node {node}: Degree {degree}")

def print_clustering_statistics(clustering_coeffs: Dict[int, float]) -> None:
    """
    Prints statistics about clustering coefficients.

    Parameters:
    - clustering_coeffs: Dictionary mapping node IDs to their clustering coefficient.
    """
    clustering_values = list(clustering_coeffs.values())
    print("\n--- Clustering Coefficient Statistics ---")
    print(f"Average Clustering Coefficient: {sum(clustering_values)/len(clustering_values):.4f}")
    print(f"Minimum Clustering Coefficient: {min(clustering_values):.4f}")
    print(f"Maximum Clustering Coefficient: {max(clustering_values):.4f}")
    print("\nTop 5 Nodes by Clustering Coefficient:")
    top_clustering = sorted(clustering_coeffs.items(), key=lambda x: x[1], reverse=True)[:5]
    for node, coeff in top_clustering:
        print(f"Node {node}: Clustering Coefficient {coeff:.4f}")

def print_centrality_statistics(centrality: Dict[int, float], method: str) -> None:
    """
    Prints statistics about a centrality measure.

    Parameters:
    - centrality: Dictionary mapping node IDs to their centrality values.
    - method: The centrality measure name.
    """
    if not centrality:
        print(f"\n--- {method.capitalize()} Centrality Statistics ---")
        print("Centrality measure could not be calculated.")
        return

    centrality_values = list(centrality.values())
    print(f"\n--- {method.capitalize()} Centrality Statistics ---")
    print(f"Average {method.capitalize()} Centrality: {sum(centrality_values)/len(centrality_values):.6f}")
    print(f"Minimum {method.capitalize()} Centrality: {min(centrality_values):.6f}")
    print(f"Maximum {method.capitalize()} Centrality: {max(centrality_values):.6f}")
    print("\nTop 5 Nodes by Centrality:")
    top_centrality = sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:5]
    for node, cent in top_centrality:
        print(f"Node {node}: {method.capitalize()} Centrality {cent:.6f}")

def print_circle_statistics(circles: Dict[str, List[int]], graph: nx.Graph) -> None:
    """
    Prints statistics about the circles (communities) in the graph.

    Parameters:
    - circles: Dictionary mapping circle names to lists of node IDs.
    - graph: The NetworkX graph.
    """
    print("\n--- Circle (Community) Statistics ---")
    print(f"Total Circles: {len(circles)}")
    circle_sizes = [len(nodes) for nodes in circles.values()]
    print(f"Average Circle Size: {sum(circle_sizes)/len(circle_sizes):.2f}")
    print(f"Minimum Circle Size: {min(circle_sizes)}")
    print(f"Maximum Circle Size: {max(circle_sizes)}")
    
    # Find circles with maximum overlapping nodes
    node_circle_counts = {}
    for circle_nodes in circles.values():
        for node in circle_nodes:
            node_circle_counts[node] = node_circle_counts.get(node, 0) + 1
    overlapping_nodes = {node: count for node, count in node_circle_counts.items() if count > 1}
    print(f"\nNodes belonging to multiple circles: {len(overlapping_nodes)}")
    if overlapping_nodes:
        print("Sample of overlapping nodes:")
        sample_overlaps = list(overlapping_nodes.items())[:5]
        for node, count in sample_overlaps:
            print(f"Node {node}: {count} circles")

def print_graph_summary(graph: nx.Graph) -> None:
    """
    Prints a summary of the graph.

    Parameters:
    - graph: The NetworkX graph.
    """
    print("\n--- Graph Summary ---")
    print(f"Number of Nodes: {graph.number_of_nodes()}")
    print(f"Number of Edges: {graph.number_of_edges()}")
    print(f"Density: {nx.density(graph):.6f}")
    connected_components = nx.number_connected_components(graph)
    print(f"Number of Connected Components: {connected_components}")
    largest_cc = max(nx.connected_components(graph), key=len)
    print(f"Size of Largest Connected Component: {len(largest_cc)}")
    print(f"Is the graph connected? {'Yes' if connected_components == 1 else 'No'}")

# 5. Main Function

def main(node_id: int):
    """
    Main function to load data, construct the graph, calculate metrics, and print all relevant information.

    Parameters:
    - node_id: The ID of the ego node.
    """
    # Define file paths based on node_id
    edges_file = f"facebook/{node_id}.edges"
    circles_file = f"facebook/{node_id}.circles"
    features_file = f"facebook/{node_id}.feat"
    ego_features_file = f"facebook/{node_id}.egofeat"
    feature_names_file = f"facebook/{node_id}.featnames"

    print(f"Loading data for Node ID: {node_id}\n")

    # Load data
    edges = load_edges(edges_file)
    circles = load_circles(circles_file)
    features = load_features(features_file)
    ego_features = load_ego_features(ego_features_file)
    feature_names = load_feature_names(feature_names_file)

    # Construct graph
    G = construct_graph(edges, ego_node=node_id)
    add_circles_to_graph(G, circles)

    # Print graph summary
    print_graph_summary(G)

    # Calculate network metrics
    degrees = calculate_degree(G)
    clustering_coeffs = calculate_clustering_coefficient(G)
    betweenness = calculate_centrality(G, method='betweenness')
    closeness = calculate_centrality(G, method='closeness')
    eigenvector = calculate_centrality(G, method='eigenvector')

    # Print degree statistics
    print_degree_statistics(degrees)

    # Print clustering coefficient statistics
    print_clustering_statistics(clustering_coeffs)

    # Print centrality statistics
    print_centrality_statistics(betweenness, 'betweenness')
    print_centrality_statistics(closeness, 'closeness')
    print_centrality_statistics(eigenvector, 'eigenvector')

    # Print circle statistics
    print_circle_statistics(circles, G)

    # Optional: Print ego node information
    print("\n--- Ego Node Information ---")
    print(f"Ego Node ID: {node_id}")
    if 'circles' in G.nodes[node_id]:
        print(f"Circles: {', '.join(G.nodes[node_id]['circles'])}")
    else:
        print("Circles: None")
    # If you want to print ego features
    # print(f"Ego Features: {ego_features}")
    # print(f"Feature Names: {feature_names}")

    # Optional: Identify bridges in the graph
    bridges = list(nx.bridges(G))
    print(f"\nNumber of Bridges in the Graph: {len(bridges)}")
    print("Sample Bridges:")
    for bridge in bridges[:5]:
        print(f"{bridge[0]} - {bridge[1]}")

    # Optional: Print top bridges based on betweenness centrality
    print("\n--- Top 5 Bridges by Betweenness Centrality ---")
    bridge_betweenness = {bridge: betweenness[bridge[0]] + betweenness[bridge[1]] for bridge in bridges}
    top_bridges = sorted(bridge_betweenness.items(), key=lambda x: x[1], reverse=True)[:5]
    for bridge, cent in top_bridges:
        print(f"{bridge[0]} - {bridge[1]}: Betweenness Centrality {cent:.6f}")

if __name__ == "__main__":
    node_id = 0  # Replace with desired node ID
    main(node_id)


Loading data for Node ID: 0


--- Graph Summary ---
Number of Nodes: 343
Number of Edges: 2519
Density: 0.042948
Number of Connected Components: 15
Size of Largest Connected Component: 324
Is the graph connected? No

--- Degree Statistics ---
Total Nodes: 343
Average Degree: 14.69
Minimum Degree: 0
Maximum Degree: 77
Median Degree: 10

Top 5 Nodes by Degree:
Node 56: Degree 77
Node 67: Degree 75
Node 271: Degree 72
Node 322: Degree 71
Node 25: Degree 68

--- Clustering Coefficient Statistics ---
Average Clustering Coefficient: 0.4934
Minimum Clustering Coefficient: 0.0000
Maximum Clustering Coefficient: 1.0000

Top 5 Nodes by Clustering Coefficient:
Node 46: Clustering Coefficient 1.0000
Node 135: Clustering Coefficient 1.0000
Node 63: Clustering Coefficient 1.0000
Node 76: Clustering Coefficient 1.0000
Node 273: Clustering Coefficient 1.0000

--- Betweenness Centrality Statistics ---
Average Betweenness Centrality: 0.007202
Minimum Betweenness Centrality: 0.000000
Maximum Betweenness 